# Como construir um sistema de recomendação para dados de compra (passo a passo)
* Descrição: uma documentação sobre a construção de modelos de filtragem colaborativa para recomendar produtos aos clientes
* Link: https://medium.com/datadriveninvestor/how-to-build-a-recommendation-system-for-purchase-data-step-by-step-d6d7a78800b6
* Autor: Moorissa Tjokro

## Declaração do problema
Neste desafio de dados, estamos construindo modelos de filtragem colaborativa para recomendar itens de produtos. As etapas a seguir têm como objetivo recomendar aos usuários seus 10 principais itens para colocar na cesta. A saída final será um arquivo csv na pasta `output` e uma função que procura uma lista de recomendações com base em um usuário especificado:
* Entrada: usuário - ID do cliente
* Retorna: lista classificada de itens (IDs de produto), que o usuário mais provavelmente deseja colocar em sua "cesta" (vazia)

## 1. Módulos de importação
* `pandas` e` numpy` para manipulação de dados
* `turicreate` para realizar a seleção e avaliação do modelo
* `sklearn` para dividir os dados no conjunto de treino e teste

In [ ]:
%load_ext autoreload
%autoreload 2
!pip install turicreate

import pandas as pd
import numpy as np
import time
import turicreate as tc
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split

#import sys
#sys.path.append("..")
#!pip install scripts
#import scripts.data_layer as data_layer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 2. Carregar dados
Dois conjuntos de dados são usados ​​neste exercício, que podem ser encontrados na pasta `data`:
* `recommend_1.csv` consistindo em uma lista de 1000 IDs de cliente para recomendar como saída
* `trx_data.csv` consistindo em transações do usuário

O formato é o seguinte.

## 3. Preparação de dados
* Nosso objetivo aqui é dividir cada lista de itens na coluna `produtos` em linhas e contar o número de produtos comprados por um usuário

In [ ]:
customerId = np.random.randint(0,1000,size=100000)
productId = np.random.randint(0,300,size=100000)
data=pd.DataFrame(list(zip(customerId, productId)),columns=['customerId','productId'])
data.sort_values
data=data.groupby(['customerId','productId']).productId.count()

data=pd.DataFrame(data)
data=data.rename({'productId':'purchase_count'},axis='columns').reset_index()

userId=list(data['customerId'].unique())

data

,customerId,productId,purchase_count
0,0,4,1
1,0,10,1
2,0,11,1
3,0,15,1
4,0,19,1
...,...,...,...
85055,999,281,1
85056,999,283,1
85057,999,290,1
85058,999,296,1


In [ ]:
data.dtypes

customerId        int64
productId         int64
purchase_count    int64
dtype: object

### 3.1. Crie dados com usuário, item e campo de destino
* Esta tabela será uma entrada para nossa modelagem posterior
    * Nesse caso, nosso usuário é `customerId`,` productId` e `purchase_count`

In [ ]:
print(data.shape)
data.head()

(85060, 3)


,customerId,productId,purchase_count
0,0,4,1
1,0,10,1
2,0,11,1
3,0,15,1
4,0,19,1


In [ ]:
data.dtypes
#pd.isna(data['customerID'])

customerId        int64
productId         int64
purchase_count    int64
dtype: object

### 3.2. Criar dummy
* Dummy para marcar se um cliente comprou aquele item ou não.
* Se alguém compra um item, `purchase_dummy` é marcado como 1
* Por que criar um dummy em vez de normalizá-lo?
    * Normalizar a contagem de compras, digamos por cada usuário, não funcionaria porque os clientes podem ter frequências de compra diferentes e não têm o mesmo gosto
    * No entanto, podemos normalizar os itens por frequência de compra para todos os usuários, o que é feito na seção 3.3. abaixo de.

In [ ]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy.loc[data_dummy['purchase_count'] != 0, 'purchase_dummy'] = 1 
    #data_dummy.query('purchase_count!=0')
    #data_dummy['purchase_dummy'] = 1
    return data_dummy
#def create_data_dummy(data):
#    data_dummy = data.copy()
#    data_dummy['purchase_dummy'] = 1
#    return data_dummy

In [ ]:
data_dummy = create_data_dummy(data)
data_dummy

,customerId,productId,purchase_count,purchase_dummy
0,0,4,1,1.0
1,0,10,1,1.0
2,0,11,1,1.0
3,0,15,1,1.0
4,0,19,1,1.0
...,...,...,...,...
85055,999,281,1,1.0
85056,999,283,1,1.0
85057,999,290,1,1.0
85058,999,296,1,1.0


### 3.3. Normalizar os valores dos itens entre os usuários
* Para fazer isso, normalizamos a frequência de compra de cada item entre os usuários, primeiro criando uma matriz de item de usuário da seguinte maneira

In [ ]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
df_matrix

productId,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
customerId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,2.0,1.0,NaN,1.0,NaN,NaN,1.0,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,2.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,1.0,NaN,1.0,NaN
1,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,1.0,1.0,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0,1.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,1.0,2.0,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,2.0,NaN,NaN,1.0,NaN,NaN
2,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,NaN,1.0
3,NaN,2.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,1.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,1.0,1.0
4,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0,NaN,NaN,1.0,NaN,NaN,1.0,1.0,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,1.0,1.0,1.0,NaN,NaN,2.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,1.0,2.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,1.0,1.0,NaN,1.0,1.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,2.0
996,1.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0,1.0,NaN,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0,2.0,NaN,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,2.0,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0
997,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0


In [ ]:
(df_matrix.shape)

(1000, 300)

In [ ]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
print(df_matrix_norm.shape)
df_matrix_norm

(1000, 300)


productId,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
customerId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.5,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,0.5,NaN,NaN,0.0,NaN,NaN,0.000000,NaN,0.0,NaN,0.0,NaN
1,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,0.000000,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,0.5,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,0.0,0.5,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.333333,NaN,NaN,0.0,NaN,NaN
2,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.333333,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.000000,NaN,0.0,NaN,NaN,0.0
3,NaN,0.333333,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,0.5,NaN,0.0,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0
4,NaN,0.000000,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.333333,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.5,NaN,0.000000,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.5,NaN,0.000000,NaN,NaN,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,NaN,0.000000,NaN,NaN,0.0,NaN,NaN,NaN,0.000000,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.5
996,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,0.5,NaN,...,NaN,NaN,0.0,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,0.333333,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.5,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0
997,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.5,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0


In [ ]:
# create a table for input to the modeling

d = df_matrix_norm.reset_index()
d.index.names = ['scaled_purchase_freq']
data_norm = pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()
print(data_norm.shape)
data_norm

(85209, 3)


,customerId,productId,scaled_purchase_freq
3,3,0,0.5
6,6,0,0.0
8,8,0,0.0
10,10,0,0.0
23,23,0,0.0
...,...,...,...
299976,976,299,0.0
299980,980,299,0.0
299984,984,299,0.0
299986,986,299,0.5


#### Defina uma função para normalizar dados




In [ ]:
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()

* Podemos normalizar o histórico de compras, de 0 a 1 (com 1 sendo o maior número de compras para um item e 0 sendo 0 contagem de compras para aquele item).

## 4. Divida o conjunto de dados em treino e teste
* Dividir os dados em conjuntos de treinamento e teste é uma parte importante da avaliação da modelagem preditiva, neste caso, um modelo de filtragem colaborativa. Normalmente, usamos uma parte maior dos dados para treinamento e uma parte menor para teste.
* Usamos a proporção 80:20 para o tamanho do nosso conjunto de treino e teste.
* Nossa parte de treinamento será usada para desenvolver um modelo preditivo, enquanto a outra para avaliar o desempenho do modelo.
* Agora que temos três conjuntos de dados com contagens de compra, compras dummy e contagens de compra normalizadas, gostaríamos de dividir cada um.

In [ ]:
train, test = train_test_split(data, test_size = .2)
print(train.shape, test.shape)

(68167, 3) (17042, 3)


In [ ]:
# Using turicreate library, we convert dataframe to SFrame - this will be useful in the modeling part

train_data = tc.SFrame(train)
test_data = tc.SFrame(test)

In [ ]:
train_data

customerId,productId,purchase_count
236,173,1
163,259,1
860,112,1
779,126,1
835,83,1
481,24,1
86,273,1
318,283,1
995,77,1
114,192,1


In [ ]:
test_data

customerId,productId,purchase_count
907,148,1
643,135,2
711,214,1
234,11,1
255,0,1
264,79,1
940,27,1
553,161,1
16,260,2
186,122,1


#### Define a função `split_data` para dividir os dados de treino e teste

In [ ]:
# We can define a function for this step as follows

def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [ ]:
# lets try with both dummy table and scaled/normalized purchase table

train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

## 5. Modelo básico
Antes de executar uma abordagem mais complicada, como filtragem colaborativa, gostaríamos de usar um modelo básico para comparar e avaliar modelos. Esse modelo normalmente usa uma abordagem muito simples, as técnicas usadas além dessa abordagem devem ser escolhidas se mostrarem precisão e complexidade relativamente melhores.

### 5.1. Usando um modelo de popularidade como modelo básico
* O modelo de popularidade usa os itens mais populares para recomendação. Esses itens são produtos com o maior número de vendas entre os clientes.
* Usamos a biblioteca `turicreate` para executar e avaliar os modelos de filtragem básico e colaborativo abaixo
* Os dados de treinamento são usados ​​para a seleção do modelo

#### Usando contagens de compra

In [ ]:
# variables to define field names
user_id = 'customerId'
item_id = 'productId'
target = 'purchase_count'
#users_to_recommend = list(transactions[user_id])
users_to_recommend = list(userId)
n_rec = 10 # number of items to recommend
n_display = 30

In [ ]:
popularity_model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)

Preparing data set.

Data has 68167 observations with 1000 users and 300 items.

Data prepared in: 0.09789s

68167 observations to process; with 300 unique items.

In [ ]:
# Get recommendations for a list of users to recommend (from customers file)
# Printed below is head / top 30 rows for first 3 customers with 10 recommendations each

popularity_recomm = popularity_model.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm.print_rows(n_display)

recommendations finished on 1000/1000 queries. users per second: 69165.9

+------------+-----------+--------------------+------+
| customerId | productId |       score        | rank |
+------------+-----------+--------------------+------+
|     0      |     89    | 1.2606635071090047 |  1   |
|     0      |     98    | 1.2456896551724137 |  2   |
|     0      |     67    | 1.244541484716157  |  3   |
|     0      |    206    |       1.2375       |  4   |
|     0      |     48    | 1.2331838565022422 |  5   |
|     0      |    190    | 1.2320675105485233 |  6   |
|     0      |    277    | 1.2274881516587677 |  7   |
|     0      |     49    | 1.2262443438914028 |  8   |
|     0      |     97    |       1.225        |  9   |
|     0      |     74    | 1.2236842105263157 |  10  |
|     1      |     98    | 1.2456896551724137 |  1   |
|     1      |    206    |       1.2375       |  2   |
|     1      |     48    | 1.2331838565022422 |  3   |
|     1      |    257    | 1.2327586206896552 |  4   |
|     1      |    216    | 1.2324561403508771 |  5   |
|     1   

#### Definir uma função `model` para a seleção do modelo

In [ ]:
# Since turicreate is very accessible library, we can define a model selection function as below

def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

In [ ]:
# variables to define field names
# constant variables include:
user_id = 'customerId'
item_id = 'productId'
users_to_recommend = list(userId)
n_rec = 10 # number of items to recommend
n_display = 30 # to print the head / first few rows in a defined dataset

#### Usando dummy

In [ ]:
# these variables will change accordingly
name = 'popularity'
target = 'purchase_dummy'
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 68167 observations with 1000 users and 300 items.

Data prepared in: 0.101046s

68167 observations to process; with 300 unique items.

recommendations finished on 1000/1000 queries. users per second: 126839

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|     0      |    127    |  1.0  |  1   |
|     0      |     12    |  1.0  |  2   |
|     0      |     45    |  1.0  |  3   |
|     0      |    245    |  1.0  |  4   |
|     0      |    272    |  1.0  |  5   |
|     0      |     89    |  1.0  |  6   |
|     0      |    120    |  1.0  |  7   |
|     0      |    281    |  1.0  |  8   |
|     0      |    255    |  1.0  |  9   |
|     0      |    156    |  1.0  |  10  |
|     1      |     45    |  1.0  |  1   |
|     1      |     79    |  1.0  |  2   |
|     1      |    245    |  1.0  |  3   |
|     1      |    272    |  1.0  |  4   |
|     1      |    198    |  1.0  |  5   |
|     1      |     35    |  1.0  |  6   |
|     1      |    120    |  1.0  |  7   |
|     1      |    281    |  1.0  |  8   |
|     1      |    255    |  1.0  |  9   |
|     1      |     9     |  1.0  |  10  |
|     2      |     12    |  1.0  |

#### Usando contagem normalizada

In [ ]:
name = 'popularity'
target = 'scaled_purchase_freq'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 68167 observations with 1000 users and 300 items.

Data prepared in: 0.080251s

68167 observations to process; with 300 unique items.

recommendations finished on 1000/1000 queries. users per second: 132890

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|     0      |    292    | 0.18552036199095023 |  1   |
|     0      |    248    | 0.15639810426540285 |  2   |
|     0      |    186    | 0.14018691588785046 |  3   |
|     0      |     89    |  0.1325581395348837 |  4   |
|     0      |     96    | 0.11572052401746726 |  5   |
|     0      |    148    | 0.11320754716981132 |  6   |
|     0      |    129    | 0.11084905660377359 |  7   |
|     0      |     69    |  0.1087866108786611 |  8   |
|     0      |    215    | 0.10869565217391304 |  9   |
|     0      |    247    |  0.1084070796460177 |  10  |
|     1      |    292    | 0.18552036199095023 |  1   |
|     1      |    248    | 0.15639810426540285 |  2   |
|     1      |    148    | 0.11320754716981132 |  3   |
|     1      |    129    | 0.11084905660377359 |  4   |
|     1      |     2     |  0.1087866108786611 |

#### Notas
* Depois de criar o modelo, previmos os itens de recomendação usando pontuações por popularidade. Como você pode ver para cada resultado do modelo acima, as linhas mostram os primeiros 30 registros de 1000 usuários com 10 recomendações. Esses 30 registros incluem 3 usuários e seus itens recomendados, junto com a pontuação e as classificações decrescentes.
* No resultado, embora diferentes modelos tenham diferentes listas de recomendações, a cada usuário é recomendada a mesma lista de 10 itens. Isso ocorre porque a popularidade é calculada considerando os itens mais populares de todos os usuários.
* Se for um exemplo de agrupamento abaixo, os produtos 132, 248, 37 e 34 são os mais populares (mais vendidos) entre os clientes. Usando suas contagens de compras divididas pelo número de clientes, vemos que esses produtos são comprados pelo menos 3 vezes em média no conjunto de transações de treinamento (o mesmo que a primeira medida de popularidade na variável `compra_conta`)

In [ ]:
train.groupby(by=item_id)['purchase_count'].mean().sort_values(ascending=False).head(20)

productId
89     1.260664
98     1.245690
67     1.244541
206    1.237500
48     1.233184
257    1.232759
216    1.232456
190    1.232068
277    1.227488
49     1.226244
97     1.225000
2      1.224215
74     1.223684
235    1.222707
35     1.222222
120    1.221739
96     1.219178
117    1.217822
139    1.217195
232    1.213675
Name: purchase_count, dtype: float64

## 6. Modelo de Filtragem Colaborativa

* Na filtragem colaborativa, recomendamos itens com base em como usuários semelhantes compram itens. Por exemplo, se o cliente 1 e o cliente 2 compraram itens semelhantes, por exemplo, 1 comprei X, Y, Z e 2 comprei X, Y, recomendaríamos um item Z ao cliente 2.

* Para definir a similaridade entre os usuários, usamos as seguintes etapas:
    1. Crie uma matriz de item de usuário, onde os valores de índice representam IDs de clientes exclusivos e os valores de coluna representam IDs de produtos exclusivos
    
    2. Crie uma matriz de similaridade item a item. A ideia é calcular a semelhança de um produto com outro produto. Existem várias maneiras de calcular isso. Nas etapas 6.1 e 6.2, usamos co-seno e medida de similaridade de Pearson, respectivamente.
    
        * Para calcular a similaridade entre os produtos X e Y, observe todos os clientes que avaliaram esses dois itens. Por exemplo, tanto X quanto Y foram avaliados pelos clientes 1 e 2.
        * Em seguida, criamos dois vetores de itens, v1 para o item X e v2 para o item Y, no espaço do usuário de (1, 2) e, a seguir, encontramos o ângulo / distância `cosseno` ou` pearson` entre esses vetores. Um ângulo zero ou vetores sobrepostos com valor de cosseno de 1 significam similaridade total (ou por usuário, em todos os itens, há a mesma classificação) e um ângulo de 90 graus significaria cosseno de 0 ou nenhuma similaridade.
        
    3. Para cada cliente, prevemos sua probabilidade de comprar um produto (ou suas contagens de compra) para produtos que ele não comprou.
    
        * Para o nosso exemplo, calcularemos a classificação para o usuário 2 no caso do item Z (item alvo). Para calcular isso, pesamos a medida de similaridade calculada entre o item de destino e outros itens que o cliente já comprou. O fator de pesagem são as contagens de compras fornecidas pelo usuário aos itens já adquiridos por ele.
        * Em seguida, normalizamos essa soma ponderada com a soma das medidas de similaridade para que a classificação calculada permaneça dentro dos limites predefinidos. Assim, a classificação prevista para o item Z para o usuário 2 seria calculada usando medidas de similaridade.

* Embora eu tenha escrito scripts Python para todo o processo, incluindo encontrar semelhanças usando scripts Python (que podem ser encontrados na pasta `scripts`, podemos usar a biblioteca` turicreate` por enquanto para capturar diferentes medidas, como usar distância de `cosine` e` pearson` e avalie o melhor modelo.

### 6.1. Similaridade `Cosine`
* Similaridade é o cosseno do ângulo entre os 2 vetores dos vetores dos itens de A e B
* É definido pela seguinte fórmula
![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTnRHSAx1c084UXF2wIHYwaHJLmq2qKtNk_YIv3RjHUO00xwlkt)
* Quanto mais próximos os vetores, menor será o ângulo e maior será o cosseno


#### Usando contagem de compra

In [ ]:
# these variables will change accordingly
name = 'cosine'
target = 'purchase_count'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 68167 observations with 1000 users and 300 items.

Data prepared in: 0.073159s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.095445s

recommendations finished on 1000/1000 queries. users per second: 20482.1

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|     0      |    285    | 0.15119367081021506 |  1   |
|     0      |    199    |  0.1482564996159266 |  2   |
|     0      |     24    | 0.12612628085272654 |  3   |
|     0      |     21    | 0.11882780847095308 |  4   |
|     0      |    244    |  0.1161845638638451 |  5   |
|     0      |     72    |  0.1115793888531034 |  6   |
|     0      |    209    | 0.10962804536970835 |  7   |
|     0      |    252    | 0.10878795858413454 |  8   |
|     0      |    123    |  0.1032911425545102 |  9   |
|     0      |    291    | 0.10154196099629478 |  10  |
|     1      |     24    |  0.1234254240989685 |  1   |
|     1      |    285    | 0.11901498089234035 |  2   |
|     1      |    230    | 0.11254894816213185 |  3   |
|     1      |    256    | 0.10425564895073573 |  4   |
|     1      |     81    | 0.10347718993822734 |

#### Usando dummy

In [ ]:
# these variables will change accordingly
name = 'cosine'
target = 'purchase_dummy'
cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 68167 observations with 1000 users and 300 items.

Data prepared in: 0.095681s

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.117101s

recommendations finished on 1000/1000 queries. users per second: 23153.5

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|     0      |    199    | 0.13650397459665933 |  1   |
|     0      |    285    | 0.13346729581318204 |  2   |
|     0      |     97    | 0.12016181813346015 |  3   |
|     0      |    291    |  0.114670059037587  |  4   |
|     0      |    206    | 0.10848072123905969 |  5   |
|     0      |     47    | 0.10741552852448963 |  6   |
|     0      |    120    | 0.10307254772337657 |  7   |
|     0      |     81    | 0.09848129465466454 |  8   |
|     0      |    189    | 0.09753868220344422 |  9   |
|     0      |    237    | 0.09719604537600562 |  10  |
|     1      |    228    | 0.11300223704540369 |  1   |
|     1      |    285    | 0.11205700401103857 |  2   |
|     1      |    298    |  0.111067227341912  |  3   |
|     1      |     81    |  0.1093567916841218 |  4   |
|     1      |     16    | 0.10819086974317377 |

#### Usando contagem de compra normalizada

In [ ]:
name = 'cosine'
target = 'scaled_purchase_freq'
cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 68167 observations with 1000 users and 300 items.

Data prepared in: 0.081772s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.085567s

recommendations finished on 1000/1000 queries. users per second: 15914.4

+------------+-----------+-----------------------+------+
| customerId | productId |         score         | rank |
+------------+-----------+-----------------------+------+
|     0      |    252    | 0.0038509689160247346 |  1   |
|     0      |    215    |  0.00350037973318527  |  2   |
|     0      |    185    |  0.003260620494387043 |  3   |
|     0      |    281    |  0.003202198156669958 |  4   |
|     0      |    291    |  0.003182673632208981 |  5   |
|     0      |     10    |  0.00299606483374069  |  6   |
|     0      |     59    | 0.0029529511038936785 |  7   |
|     0      |     84    | 0.0029098542768563797 |  8   |
|     0      |    212    | 0.0028771642428725513 |  9   |
|     0      |     86    |  0.002860177808733129 |  10  |
|     1      |    226    |  0.002615955242743859 |  1   |
|     1      |    150    |  0.002438635092515212 |  2   |
|     1      |     50    | 0.0023606557112473706 |  3   |
|     1      |    209    | 0.0023601009295536922 |  4   |
|     1      |

### 6.2. Similaridade de `Pearson`
* Similaridade é o coeficiente de Pearson entre os dois vetores.
* É definido pela seguinte fórmula
![](http://critical-numbers.group.shef.ac.uk/glossary/images/correlationKT1.png)

#### Usando contagem de compra

In [ ]:
# these variables will change accordingly
name = 'pearson'
target = 'purchase_count'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 68167 observations with 1000 users and 300 items.

Data prepared in: 0.074786s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.132988s

recommendations finished on 1000/1000 queries. users per second: 15865.5

+------------+-----------+--------------------+------+
| customerId | productId |       score        | rank |
+------------+-----------+--------------------+------+
|     0      |     89    | 1.2601182409802838 |  1   |
|     0      |     98    | 1.2460126612574014 |  2   |
|     0      |     67    | 1.244328589224391  |  3   |
|     0      |    206    | 1.2361412733320216 |  4   |
|     0      |     48    | 1.2350543921711146 |  5   |
|     0      |    190    | 1.2307095577948208 |  6   |
|     0      |     49    | 1.2281342453931947 |  7   |
|     0      |    277    | 1.227186140600324  |  8   |
|     0      |     74    | 1.2246122311928314 |  9   |
|     0      |     97    | 1.2240486398575798 |  10  |
|     1      |     98    | 1.2426413155207239 |  1   |
|     1      |    206    | 1.2359628674056804 |  2   |
|     1      |    257    | 1.232364415437325  |  3   |
|     1      |     48    | 1.2320337492840823 |  4   |
|     1      |    190    | 1.2309943661722458 |  5   |
|     1   

#### Usando compra dummy

In [ ]:
# these variables will change accordingly
name = 'pearson'
target = 'purchase_dummy'
pear_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 68167 observations with 1000 users and 300 items.

Data prepared in: 0.085401s

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.162826s

recommendations finished on 1000/1000 queries. users per second: 88597.5

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|     0      |    127    |  0.0  |  1   |
|     0      |     12    |  0.0  |  2   |
|     0      |     45    |  0.0  |  3   |
|     0      |    245    |  0.0  |  4   |
|     0      |    272    |  0.0  |  5   |
|     0      |     89    |  0.0  |  6   |
|     0      |    120    |  0.0  |  7   |
|     0      |    281    |  0.0  |  8   |
|     0      |    255    |  0.0  |  9   |
|     0      |    156    |  0.0  |  10  |
|     1      |     45    |  0.0  |  1   |
|     1      |     79    |  0.0  |  2   |
|     1      |    245    |  0.0  |  3   |
|     1      |    272    |  0.0  |  4   |
|     1      |    198    |  0.0  |  5   |
|     1      |     35    |  0.0  |  6   |
|     1      |    120    |  0.0  |  7   |
|     1      |    281    |  0.0  |  8   |
|     1      |    255    |  0.0  |  9   |
|     1      |     9     |  0.0  |  10  |
|     2      |     12    |  0.0  |

#### Usando contagem normalizada de compra

In [ ]:
name = 'pearson'
target = 'scaled_purchase_freq'
pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 68167 observations with 1000 users and 300 items.

Data prepared in: 0.081199s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.166367s

recommendations finished on 1000/1000 queries. users per second: 21504

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|     0      |    292    | 0.18541891933365026 |  1   |
|     0      |    248    |  0.1558179411210493 |  2   |
|     0      |    186    | 0.14059029877277288 |  3   |
|     0      |     89    |  0.1327150990258733 |  4   |
|     0      |     96    | 0.11546410394602201 |  5   |
|     0      |    148    | 0.11305506603377409 |  6   |
|     0      |    129    | 0.11068171256026962 |  7   |
|     0      |    215    |  0.109349855852158  |  8   |
|     0      |     69    | 0.10890544118942569 |  9   |
|     0      |    247    | 0.10858608040087223 |  10  |
|     1      |    292    | 0.18568628298211415 |  1   |
|     1      |    248    | 0.15565195956718653 |  2   |
|     1      |    148    | 0.11323463977334819 |  3   |
|     1      |    129    | 0.11081795856810792 |  4   |
|     1      |     2     | 0.10947096573412521 |

#### Observação
* Na filtragem colaborativa acima, usamos duas abordagens: cosseno e distância de Pearson. Também conseguimos aplicá-los a três conjuntos de dados de treinamento com contagens normais, dummy ou contagens normalizadas de compra de itens.
* Podemos ver que as recomendações são diferentes para cada usuário. Isso sugere que a personalização existe.
* Mas quão bom é este modelo em comparação com a linha de base e entre si? Precisamos de alguns meios de avaliar um mecanismo de recomendação. Vamos nos concentrar nisso na próxima seção.

## 7. Avaliação do modelo
Para avaliar os mecanismos de recomendação, podemos usar o conceito de recall de precisão.

* RMSE (erros quadráticos médios)
    * Mede o erro dos valores previstos
    * Quanto menor o valor RMSE, melhores as recomendações
* Lembrar
    * Qual porcentagem de produtos que um usuário compra são realmente recomendados?
    * Se um cliente comprar 5 produtos e a recomendação decidir mostrar 3 deles, o recall é de 0,6
* Precisão
    * De todos os itens recomendados, quantos o usuário realmente gostou?
    * Se 5 produtos foram recomendados ao cliente, dos quais ele compra 4 deles, a precisão é 0,8
    
* Por que o recall e a precisão são importantes?
    * Considere um caso em que recomendamos todos os produtos, para que nossos clientes com certeza tenha todos os itens que gostaria de compraram. Nesse caso, temos 100% de recall! Isso significa que nosso modelo é bom?
    * Temos que considerar a precisão. Se recomendarmos 300 itens, mas o usuário gostar e comprar apenas 3 deles, a precisão é de 0,1%! Essa precisão muito baixa indica que o modelo não é ótimo, apesar de seu excelente recall.
    * Portanto, nosso objetivo deve ser otimizar o recall e a precisão (para ser o mais próximo possível de 1).

Vamos comparar todos os modelos que construímos com base nas características de recuperação de precisão:

In [ ]:
# create initial callable variables

models_w_counts = [popularity_model, cos, pear]
models_w_dummy = [pop_dummy, cos_dummy, pear_dummy]
models_w_norm = [pop_norm, cos_norm, pear_norm]

names_w_counts = ['Popularity Model on Purchase Counts', 'Cosine Similarity on Purchase Counts', 'Pearson Similarity on Purchase Counts']
names_w_dummy = ['Popularity Model on Purchase Dummy', 'Cosine Similarity on Purchase Dummy', 'Pearson Similarity on Purchase Dummy']
names_w_norm = ['Popularity Model on Scaled Purchase Counts', 'Cosine Similarity on Scaled Purchase Counts', 'Pearson Similarity on Scaled Purchase Counts']

#### Modelos para contagem de compra

In [ ]:
eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)

PROGRESS: Evaluate model Popularity Model on Purchase Counts


recommendations finished on 1000/1000 queries. users per second: 69175.4


Precision and recall summary statistics by cutoff
+--------+---------------------+----------------------+
| cutoff |    mean_precision   |     mean_recall      |
+--------+---------------------+----------------------+
|   1    |        0.074        | 0.004553496983055134 |
|   2    | 0.07050000000000002 | 0.008534334590804164 |
|   3    |         0.07        | 0.012473516311745715 |
|   4    | 0.07249999999999997 | 0.01727320685751109  |
|   5    | 0.07200000000000001 | 0.021478344986085078 |
|   6    | 0.07350000000000004 | 0.02640825137316505  |
|   7    | 0.07399999999999997 | 0.03101538199239251  |
|   8    | 0.07612499999999997 | 0.03630766927343233  |
|   9    | 0.07588888888888888 | 0.040712873179943945 |
|   10   | 0.07629999999999999 | 0.04546923518986109  |
+--------+---------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.42602247064701276

Per User RMSE (best)
+------------+---------------------+-------+
| customerId |         rmse        | cou

recommendations finished on 1000/1000 queries. users per second: 19181


Precision and recall summary statistics by cutoff
+--------+---------------------+-----------------------+
| cutoff |    mean_precision   |      mean_recall      |
+--------+---------------------+-----------------------+
|   1    |        0.073        | 0.0042639042951845806 |
|   2    |        0.073        |  0.00859128471363243  |
|   3    | 0.07233333333333337 |  0.01289708587830609  |
|   4    |        0.0695       |  0.01656853003781813  |
|   5    | 0.07080000000000004 |  0.021060178017990646 |
|   6    | 0.07033333333333336 |  0.025102629961061114 |
|   7    | 0.06942857142857146 |  0.02880485364188739  |
|   8    |       0.069125      |  0.032787384975587114 |
|   9    |  0.0696666666666667 |  0.037339577101934876 |
|   10   | 0.07109999999999998 |  0.042127696019929335 |
+--------+---------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 1.192735026094086

Per User RMSE (best)
+------------+-------------------+-------+
| customerId |        rmse    

recommendations finished on 1000/1000 queries. users per second: 19894.6


Precision and recall summary statistics by cutoff
+--------+---------------------+----------------------+
| cutoff |    mean_precision   |     mean_recall      |
+--------+---------------------+----------------------+
|   1    | 0.07100000000000001 | 0.004329717521775672 |
|   2    | 0.07399999999999993 | 0.008963241180430568 |
|   3    | 0.07066666666666672 | 0.012603809994670978 |
|   4    | 0.07075000000000006 | 0.016746238402214127 |
|   5    | 0.07280000000000005 | 0.021761750413422758 |
|   6    | 0.07366666666666673 | 0.026476831010537594 |
|   7    |  0.0744285714285714 | 0.031164968065933697 |
|   8    |       0.07525       | 0.03587686563584108  |
|   9    |  0.0763333333333334 | 0.04089099316510726  |
|   10   | 0.07660000000000002 | 0.045578094825872766 |
+--------+---------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.4260051104623363

Per User RMSE (best)
+------------+-------------------+-------+
| customerId |        rmse       | count |


#### Modelo de compra dummy

In [ ]:
eval_dummy = tc.recommender.util.compare_models(test_data_dummy, models_w_dummy, model_names=names_w_dummy)

PROGRESS: Evaluate model Popularity Model on Purchase Dummy


recommendations finished on 1000/1000 queries. users per second: 75369.3


Precision and recall summary statistics by cutoff
+--------+---------------------+----------------------+
| cutoff |    mean_precision   |     mean_recall      |
+--------+---------------------+----------------------+
|   1    |        0.077        | 0.004629329766256339 |
|   2    | 0.07500000000000004 | 0.008975205469550692 |
|   3    | 0.07066666666666668 | 0.012752158705750222 |
|   4    |        0.073        | 0.017327524397251992 |
|   5    | 0.07400000000000001 | 0.021918418079468545 |
|   6    | 0.07350000000000004 | 0.025979032100324956 |
|   7    | 0.07471428571428573 | 0.030673014557309503 |
|   8    | 0.07387500000000002 | 0.03484220379863102  |
|   9    | 0.07422222222222223 | 0.039096988836073945 |
|   10   | 0.07379999999999996 | 0.043250374436765715 |
+--------+---------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.0

Per User RMSE (best)
+------------+------+-------+
| customerId | rmse | count |
+------------+------+-------+
|    918   

recommendations finished on 1000/1000 queries. users per second: 17890.4


Precision and recall summary statistics by cutoff
+--------+---------------------+----------------------+
| cutoff |    mean_precision   |     mean_recall      |
+--------+---------------------+----------------------+
|   1    | 0.08500000000000002 | 0.004946886222017461 |
|   2    | 0.09149999999999998 | 0.010957647210492225 |
|   3    | 0.08233333333333342 | 0.014688904861846545 |
|   4    | 0.08075000000000003 | 0.019106884355980163 |
|   5    | 0.07820000000000008 | 0.023138392883112018 |
|   6    | 0.07966666666666661 | 0.028313382916684636 |
|   7    | 0.07928571428571425 | 0.032742769862379284 |
|   8    | 0.07737499999999994 | 0.03643712768968277  |
|   9    | 0.07799999999999996 | 0.04121565745160059  |
|   10   | 0.07790000000000004 | 0.045761879043271635 |
+--------+---------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.9468731324045435

Per User RMSE (best)
+------------+--------------------+-------+
| customerId |        rmse        | count 

recommendations finished on 1000/1000 queries. users per second: 70561.7


Precision and recall summary statistics by cutoff
+--------+---------------------+----------------------+
| cutoff |    mean_precision   |     mean_recall      |
+--------+---------------------+----------------------+
|   1    |        0.077        | 0.004629329766256335 |
|   2    | 0.07500000000000001 | 0.008975205469550704 |
|   3    | 0.07066666666666664 | 0.012752158705750227 |
|   4    | 0.07300000000000005 | 0.017327524397252006 |
|   5    |        0.074        | 0.021918418079468538 |
|   6    | 0.07350000000000004 | 0.025979032100324945 |
|   7    |  0.0747142857142857 | 0.03067301455730951  |
|   8    | 0.07387500000000002 | 0.03484220379863103  |
|   9    | 0.07422222222222224 | 0.03909698883607395  |
|   10   | 0.07379999999999995 | 0.04325037443676571  |
+--------+---------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 1.0

Per User RMSE (best)
+------------+------+-------+
| customerId | rmse | count |
+------------+------+-------+
|    918   

#### Modelos de frequencia normalizado

In [ ]:
eval_norm = tc.recommender.util.compare_models(test_data_norm, models_w_norm, model_names=names_w_norm)

PROGRESS: Evaluate model Popularity Model on Scaled Purchase Counts


recommendations finished on 1000/1000 queries. users per second: 68785.3


Precision and recall summary statistics by cutoff
+--------+---------------------+----------------------+
| cutoff |    mean_precision   |     mean_recall      |
+--------+---------------------+----------------------+
|   1    | 0.06700000000000002 | 0.003797393049087084 |
|   2    |        0.077        | 0.008951535414837671 |
|   3    | 0.07400000000000001 | 0.012941994087609236 |
|   4    | 0.07450000000000002 |  0.0175476804221339  |
|   5    | 0.07299999999999998 | 0.021508740205592396 |
|   6    | 0.07400000000000005 | 0.026114388689126536 |
|   7    | 0.07542857142857146 | 0.031092979095702234 |
|   8    | 0.07412500000000001 | 0.03487174635938332  |
|   9    | 0.07255555555555555 | 0.03829145727286229  |
|   10   | 0.07250000000000001 | 0.04249883812717742  |
+--------+---------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.1941117593694544

Per User RMSE (best)
+------------+---------------------+-------+
| customerId |         rmse        | coun

recommendations finished on 1000/1000 queries. users per second: 21295.6


Precision and recall summary statistics by cutoff
+--------+---------------------+----------------------+
| cutoff |    mean_precision   |     mean_recall      |
+--------+---------------------+----------------------+
|   1    | 0.07500000000000005 | 0.004435866107900362 |
|   2    | 0.07300000000000004 | 0.008433625373955492 |
|   3    | 0.07366666666666664 | 0.01278802506076903  |
|   4    | 0.07475000000000001 | 0.017385644887435835 |
|   5    | 0.07500000000000004 | 0.02176102471668464  |
|   6    | 0.07499999999999998 | 0.02590313753416628  |
|   7    | 0.07585714285714287 | 0.03054416120496957  |
|   8    | 0.07587499999999998 | 0.03528227746467891  |
|   9    | 0.07677777777777772 | 0.04015157159342249  |
|   10   | 0.07670000000000002 | 0.044682111919782226 |
+--------+---------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.20897470856608272

Per User RMSE (best)
+------------+-----------------------+-------+
| customerId |          rmse         |

recommendations finished on 1000/1000 queries. users per second: 18726.2


Precision and recall summary statistics by cutoff
+--------+---------------------+-----------------------+
| cutoff |    mean_precision   |      mean_recall      |
+--------+---------------------+-----------------------+
|   1    | 0.06700000000000005 | 0.0037973930490870856 |
|   2    | 0.07700000000000001 |  0.008951535414837671 |
|   3    |        0.074        |  0.012941994087609234 |
|   4    | 0.07475000000000001 |  0.01763154794350143  |
|   5    | 0.07380000000000003 |  0.021710341715761687 |
|   6    | 0.07516666666666662 |  0.026437563685149483 |
|   7    | 0.07457142857142855 |  0.030670511809171127 |
|   8    | 0.07349999999999994 |  0.034334664068422366 |
|   9    | 0.07244444444444448 |  0.03803211304983382  |
|   10   |        0.0715       |   0.041852247131314   |
+--------+---------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.1941076266956332

Per User RMSE (best)
+------------+---------------------+-------+
| customerId |         rmse

## 8. Seleção de modelo
### 8.1. Resumo da avaliação
* Com base em RMSE


    1. Popularidade nas contagens de compra: 1,27
    2. Similaridade de cosseno nas contagens de compra: 1,67
    3. Similaridade de Pearson nas contagens de compra: 1,27
    
    4. Popularidade no dummy de compra: 0
    5. Similaridade de cosseno no dummy de compra: 0,3
    6. Similaridade de Pearson no dummy de compra: 1
    
    7. Popularidade em contagens de compra em normalizada: 1,19
    8. Similaridade de cosseno em contagens de compra em normalizada:0,23
    9. Similaridade de Pearson nas contagens de compra em normalizada: 0,19
    
    
* Com base na precisão e recall
![](../images/model_comparisons.png)


#### Notas

* Popularidade v. Filtragem colaborativa: podemos ver que os algoritmos de filtragem colaborativa funcionam melhor do que o modelo de popularidade para contagens de compras. Na verdade, o modelo de popularidade não oferece nenhuma personalização, uma vez que apenas fornece a mesma lista de itens recomendados para todos os usuários.
* Precisão e recall: Observando o resumo acima, vemos que a precisão e o recall para Contagens de compra> Dummy de compra> Contagens de compra normalizadas. No entanto, como as pontuações de recomendação para os dados de compra normalizados são zero e constantes, escolhemos a dummy. Na verdade, o RMSE não é muito diferente entre os modelos do dummy e os dos dados normalizados.
* RMSE: Como o RMSE é maior usando a distância de Pearson do que o cosseno, escolheríamos modelar os erros quadrados médios menores, que neste caso seriam o cosseno.
Portanto, selecionamos a abordagem de similaridade de cosseno no dummy de compra como nosso modelo final.

## 8. Resultado final
* Nesta etapa, gostaríamos de manipular o formato da saída de recomendação para um que possamos exportar para csv e também uma função que retornará a lista de recomendações com um ID de cliente.
* Precisamos primeiro executar novamente o modelo usando todo o conjunto de dados, pois chegamos a um modelo final usando dados de trem e avaliados com conjunto de teste.

In [ ]:
users_to_recommend = list(userId)

final_model = tc.item_similarity_recommender.create(tc.SFrame(data_dummy), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='purchase_dummy', 
                                            similarity_type='cosine')

recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

len(users_to_recommend)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 85209 observations with 1000 users and 300 items.

Data prepared in: 0.093231s

...

Generating candidate set for working with new users.

Finished training in 0.108477s

recommendations finished on 1000/1000 queries. users per second: 16104.1

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|     0      |    199    |  0.1860223955068833 |  1   |
|     0      |    285    | 0.18230236417207962 |  2   |
|     0      |     59    | 0.16750603532179809 |  3   |
|     0      |    291    |  0.1631028950214386 |  4   |
|     0      |    209    | 0.15226318362431648 |  5   |
|     0      |     73    |  0.1431885506862249 |  6   |
|     0      |    252    | 0.13812870780626932 |  7   |
|     0      |    149    | 0.13707578487885305 |  8   |
|     0      |     24    |  0.1318174470693637 |  9   |
|     0      |    189    | 0.13066580891609192 |  10  |
|     1      |     20    | 0.14848295099595013 |  1   |
|     1      |    168    | 0.14637334066278795 |  2   |
|     1      |     12    | 0.13360682725906373 |  3   |
|     1      |     59    | 0.13244949228623334 |  4   |
|     1      |     24    | 0.13199807475594913 |

1000

### 8.1. CSV output file

In [ ]:
df_rec = recom.to_dataframe()
print(df_rec.shape)
df_rec.head(30)

(10000, 4)


,customerId,productId,score,rank
0,0,199,0.186022,1
1,0,285,0.182302,2
2,0,59,0.167506,3
3,0,291,0.163103,4
4,0,209,0.152263,5
5,0,73,0.143189,6
6,0,252,0.138129,7
7,0,149,0.137076,8
8,0,24,0.131817,9
9,0,189,0.130666,10


In [ ]:
df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id].transform(lambda x: '|'.join(x.astype(str)))
df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates().sort_values('customerId').set_index('customerId')

#### Define função para cria a saida desejada


In [ ]:
def create_output(model, users_to_recommend, n_rec, print_csv=True):
    recomendation = model.recommend(users=users_to_recommend, k=n_rec)
    df_rec = recomendation.to_dataframe()
    df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id] \
        .transform(lambda x: '|'.join(x.astype(str)))
    df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates() \
        .sort_values('customerId').set_index('customerId')
    if print_csv:
        df_output.to_csv('/content/option1_recommendation.csv')
        print("An output file can be found in 'output' folder with name 'option1_recommendation.csv'")
    return df_output

In [ ]:
df_output = create_output(pear_norm, users_to_recommend, n_rec, print_csv=True)
print(df_output.shape)
df_output.head()

recommendations finished on 1000/1000 queries. users per second: 15383.4

An output file can be found in 'output' folder with name 'option1_recommendation.csv'
(1000, 1)


,recommendedProducts
customerId,
0,292|248|186|89|96|148|129|215|69|247
1,292|248|148|129|2|69|215|247|139|273
2,292|248|186|89|96|148|129|69|215|273
3,292|248|89|96|148|129|2|69|247|139
4,292|248|89|96|148|129|2|69|247|215


### 8.2. Função de recomendação

In [ ]:
def customer_recomendation(customer_id):
    if customer_id not in df_output.index:
        print('Customer not found.')
        return customer_id
    return df_output.loc[customer_id]

In [ ]:
customer_recomendation(4)

recommendedProducts    292|248|89|96|148|129|2|69|247|215
Name: 4, dtype: object

In [ ]:
customer_recomendation(21)

recommendedProducts    248|186|89|96|148|129|215|69|247|172
Name: 21, dtype: object

## Resumo
Neste exercício, pudemos realizar um processo passo a passo para fazer recomendações aos clientes. Usamos abordagens de Filtragem Colaborativa com as medidas `cosine` e` pearson` e comparamos os modelos com nosso modelo de popularidade de linha de base. Também preparamos três conjuntos de dados que incluem contagem de compras regular, compra dummy, bem como frequência de compra normalizada como nossa variável de destino. Usando RMSE, precisão e recall, avaliamos nossos modelos e observamos o impacto da personalização. Finalmente, selecionamos a abordagem Cosine em dados de compra dummy.